In [ ]:
import time
import os
from multiprocessing import Pool
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def data_ingest(file: str):
  return pd.read_csv('/storage/ETH_randomness/'+file, index_col=0)


In [ ]:
files = os.listdir('/storage/ETH_randomness/')

with Pool() as pool:
  df = pd.concat(pool.map(data_ingest, files))

data_ingest()

In [ ]:
sns.boxplot(x=df.chi)
plt.show()
sns.boxplot(x=df.ks)
plt.show()